## Phương pháp lọc - Các thông tin cơ bản - Các kiểm định thống kê lọc

### Kết hợp mọi thứ

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold, f_classif, SelectKBest

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge

from sklearn.metrics import r2_score

In [2]:
# load tập dữ liệu Santander customer satisfaction từ Kaggle

data = pd.read_csv('./datatset/house-price/houseprice.csv')
data = data[data.columns[data.dtypes != 'object']]
data = data.dropna()
data.shape

(1121, 38)

In [3]:
data.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,61,0,0,0,0,0,2,2008,208500
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,298,0,0,0,0,0,0,5,2007,181500
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,42,0,0,0,0,0,9,2008,223500
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,35,272,0,0,0,0,2,2006,140000
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,192,84,0,0,0,0,0,12,2008,250000


In [4]:
# hãy thay đổi một số cột để xem lab này hoạt động!

data['OverallQual'] = 7
data['3SsnPorch'] = 8
data['ScreenPorch'] = 9
data['PoolArea'] = 10

data.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,61,0,8,9,10,0,2,2008,208500
1,2,20,80.0,9600,7,8,1976,1976,0.0,978,...,298,0,0,8,9,10,0,5,2007,181500
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,42,0,8,9,10,0,9,2008,223500
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,35,272,8,9,10,0,2,2006,140000
4,5,60,84.0,14260,7,5,2000,2000,350.0,655,...,192,84,0,8,9,10,0,12,2008,250000


In [5]:
# chia thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['SalePrice'], axis=1),
    data['SalePrice'],
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((784, 37), (337, 37))

In [6]:
# giữ một bản sao tập dữ liệu với các biến
# để so sánh chất lượng mô hình học máy
# ở cuối notebook

X_train_original = X_train.copy()
X_test_original = X_test.copy()

### Loại bỏ các đặc trưng không đổi

In [7]:
## Yêu cầu 1
constant_features = [
    feat for feat in X_train.columns if X_train[feat].std() == 0
]

X_train.drop(labels=constant_features, axis=1, inplace=True)
## VIẾT CODE Ở ĐÂY:
X_test.drop(labels=constant_features, axis=1, inplace=True)

X_train.shape, X_test.shape

((784, 33), (337, 33))

### Loại bỏ các đặc trưng gần như không đổi

In [8]:
## Yêu cầu 2

## VIẾT CODE Ở ĐÂY:
sel = VarianceThreshold(threshold=.01) # đặt ngưỡng = 0.01

sel.fit(X_train) # tìm các đặc trưng với phương sai thấp

sum(sel.get_support()) # có bao nhiêu đặc trưng gần như không đổi?

33

<details><summary> Gợi ý </summary>

[VarianceThreshold()](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.VarianceThreshold.html)

</details>

In [9]:
## Yêu cầu 3
## VIẾT CODE Ở ĐÂY:
features_to_keep = X_train.columns[sel.get_support()]

<details><summary> Gợi ý </summary>

Using ```get_support()``` function

</details>

In [10]:
## Yêu cầu 4
# loại bỏ các đặc trưng

## VIẾT CODE Ở ĐÂY:
X_train = sel.transform(X_train)
X_test = sel.transform(X_test)

X_train.shape, X_test.shape

((784, 33), (337, 33))

In [11]:
# các biến đổi của sklearn dẫn tới các mảng numpy
# ở đây chúng ta biến đổi mảng thành dataframe

X_train= pd.DataFrame(X_train)
X_train.columns = features_to_keep

X_test= pd.DataFrame(X_test)
X_test.columns = features_to_keep

In [12]:
X_train.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,MiscVal,MoSold,YrSold
0,123.0,20.0,75.0,9464.0,7.0,1958.0,1958.0,135.0,570.0,0.0,...,0.0,1958.0,1.0,288.0,0.0,0.0,0.0,0.0,6.0,2008.0
1,151.0,20.0,120.0,10356.0,6.0,1975.0,1975.0,0.0,716.0,0.0,...,0.0,1975.0,2.0,440.0,0.0,0.0,0.0,0.0,1.0,2007.0
2,951.0,20.0,60.0,7200.0,8.0,1950.0,2002.0,0.0,398.0,149.0,...,0.0,1980.0,2.0,720.0,194.0,0.0,0.0,0.0,7.0,2007.0
3,47.0,50.0,48.0,12822.0,5.0,2003.0,2003.0,0.0,1351.0,0.0,...,1.0,2003.0,2.0,670.0,168.0,43.0,0.0,0.0,8.0,2009.0
4,194.0,160.0,24.0,2522.0,5.0,2004.0,2004.0,50.0,0.0,0.0,...,0.0,2004.0,2.0,380.0,0.0,40.0,0.0,0.0,5.0,2006.0


### Loại bỏ các đặc trưng trùng lặp

In [15]:
## Yêu cầu 5
# kiểm tra các đặc trưng trùng lặp trong tập huấn luyện

duplicated_feat = []
for i in range(0, len(X_train.columns)):
    if i % 10 == 0:  # điều này giúp chúng ta hiểu vòng lặp diễn ra thế nào
        print(i)

    col_1 = X_train.columns[i]

    for col_2 in X_train.columns[i + 1:]:
        if X_train[col_1].equals(X_train[col_2]):
            ## VIẾT CODE Ở ĐÂY:
            duplicated_feat.append(col_2)
            
len(duplicated_feat)

0
10
20
30


0

In [16]:
## Yêu cầu 6
# các đặc trưng trùng lặp đã loại
X_train.drop(labels=duplicated_feat, axis=1, inplace=True)
## VIẾT CODE Ở ĐÂY:
X_test.drop(labels=duplicated_feat, axis = 1,inplace = True)

X_train.shape, X_test.shape

((784, 33), (337, 33))

In [17]:
## Yêu cầu 7
# tạo một bản sao tập dữ liệu trừ các biến không đổi và các biến trùng lặp
# để đo chất lượng mô hình học máy
# ở cuối notebook

X_train_basic_filter = X_train.copy()
## VIẾT CODE Ở ĐÂY:
X_test_basic_filter = X_test.copy()

### Loại bỏ các đặc trưng tương quan

In [27]:
## Yêu cầu 8
# tìm và bỏ các đặc trưng tương quan
def correlation(dataset, threshold):
    
    col_corr = set()  # tập hợp tất cả tên của các cột tương quan
    corr_matrix = dataset.corr()
    
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            # chúng ta cần tìm giá trị hệ số tuyệt đối
            if abs(corr_matrix.iloc[i, j]) > threshold:
                colname = corr_matrix.columns[i]  # lấy tên của cột
                ## VIẾT CODE Ở ĐÂY:
                col_corr.add(colname)
    
    ## VIẾT CODE Ở ĐÂY:
    return col_corr


corr_features = correlation(X_train, 0.8)
print('correlated features: ', len(set(corr_features)))

correlated features:  0


In [28]:
corr_features

set()

In [29]:
## Yêu cầu 9
# các đặc trưng tương quan đã loại
X_train.drop(labels=corr_features, axis=1, inplace=True)
## VIẾT CODE Ở ĐÂY:
X_test.drop(labels=corr_features,axis = 1, inplace = True)

X_train.shape, X_test.shape

((784, 29), (337, 29))

In [30]:
# tạo một bản sao tập dữ liệu ở giai đoạn này
X_train_corr = X_train.copy()
X_test_corr = X_test.copy()

### Lựa chọn các đặc trưng dựa trên anova

In [31]:
## Yêu cầu 10
## VIẾT CODE Ở ĐÂY:
sel_ = SelectKBest(f_classif, k = 20).fit(X_train, y_train) # set k=20

# thu thập tên các đặc trưng đã chọn
features_to_keep = X_train.columns[sel_.get_support()]

# lựa chọn đặc trưng
X_train_anova = sel_.transform(X_train)
X_test_anova = sel_.transform(X_test)

# mảng numpy thành dataframe
X_train_anova = pd.DataFrame(X_train_anova)
X_train_anova.columns = features_to_keep

X_test_anova = pd.DataFrame(X_test_anova)
X_test_anova.columns = features_to_keep

X_train_anova.shape, X_test_anova.shape

((784, 20), (337, 20))

<details><summary> Gợi ý </summary>

[SelectKBest()](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html)

</details>

### So sánh chất lượng các thuật toán học máy

In [33]:
# tạo một hàm xây dựng rừng ngẫu nhiên và
# so sánh chất lượng của nó trong tập huấn luyện và tập kiểm tra

def run_linear(X_train, X_test, y_train, y_test):
    
    rf = Ridge()
    rf.fit(X_train, y_train)
    
    print('Train set')
    pred = rf.predict(X_train)
    print('Ridge Regression r2 score: {}'.format(r2_score(y_train, pred)))
    
    print('Test set')
    pred = rf.predict(X_test)
    print('Ridge Regression r2 score: {}'.format(r2_score(y_test, pred)))

In [34]:
## Yêu cầu 11
# ban đầu

## VIẾT CODE Ở ĐÂY:
run_linear(X_train_original, X_test_original, y_train, y_test)

Train set
Ridge Regression r2 score: 0.822414918262909
Test set
Ridge Regression r2 score: 0.5628164950897124


In [35]:
## Yêu cầu 12
# các phương pháp lọc - cơ bản

## VIẾT CODE Ở ĐÂY:
run_linear(X_train_basic_filter, X_test_basic_filter, y_train,y_test)

Train set
Ridge Regression r2 score: 0.8224149182629089
Test set
Ridge Regression r2 score: 0.5628164950897132


In [36]:
## Yêu cầu 13
# các phương pháp lọc - tương quan

## VIẾT CODE Ở ĐÂY:
run_linear(X_train_corr, X_test_corr, y_train, y_test)

Train set
Ridge Regression r2 score: 0.8183531757688285
Test set
Ridge Regression r2 score: 0.5609344929883318


In [37]:
## Yêu cầu 14
# các phương pháp lọc - anova

## VIẾT CODE Ở ĐÂY:
run_linear(X_train_anova,X_test_anova, y_train, y_test)

Train set
Ridge Regression r2 score: 0.8129428259312356
Test set
Ridge Regression r2 score: 0.5353526632680172


Chúng ta thấy loại một số đặc trưng thì mô hình vẫn tốt!